In [1]:
import tensorflow as tf

In [2]:
import numpy as np

In [3]:
import pandas as pd

## MNIST data preparation

In [4]:
mnist_data = pd.read_csv("./data/train.csv")

In [5]:
mnist_data.shape

(42000, 785)

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
train, test = train_test_split(mnist_data, test_size=0.2, shuffle=False)

In [8]:
train.shape

(33600, 785)

In [9]:
Y_train = train["label"]

# Drop 'label' column
X_train = train.drop(labels = ["label"],axis = 1) 

In [10]:
Y_test = test["label"]

# Drop 'label' column
X_test = test.drop(labels = ["label"],axis = 1) 

## DNN

In [11]:
n_inputs = 28*28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [12]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")

In [13]:
y = tf.placeholder(tf.int64, shape=(None), name="y")

In [14]:
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1",
                             activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2",
                             activation=tf.nn.relu)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")

In [15]:
with tf.name_scope("loss"):
  xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                          logits=logits)
  loss = tf.reduce_mean(xentropy, name="loss")

In [16]:
learning_rate = 0.01

In [17]:
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)

In [18]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [19]:
init = tf.global_variables_initializer()

In [20]:
saver = tf.train.Saver()

In [21]:
n_epochs = 40

In [22]:
batch_size = 50

In [23]:
def get_batch(df, batch_size):
    return df.sample(batch_size)

In [25]:
len(train)

33600

In [24]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        i = 0
        for iteration in range(len(train) // batch_size):
            batch = get_batch(train, batch_size)
            X_batch = batch.drop(labels = ["label"],axis = 1) 
            Y_batch = batch["label"]
            
            i = i + batch_size
            sess.run(training_op, feed_dict={X: X_batch, y: Y_batch})
        print(X_batch.shape)
        print(Y_batch.shape)
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: Y_batch})
        acc_val = accuracy.eval(feed_dict={X: X_test,
                                      y: Y_test})
        print(epoch, "Train accuracy:", acc_train,
             "Validation accuracy:", acc_val)
    save_path = saver.save(sess, "./dnn_model_mnist.ckpt")

(50, 784)
(50,)
0 Train accuracy: 0.0 Validation accuracy: 0.0
(50, 784)
(50,)
1 Train accuracy: 0.0 Validation accuracy: 0.0
(50, 784)
(50,)
2 Train accuracy: 0.0 Validation accuracy: 0.0
(50, 784)
(50,)
3 Train accuracy: 0.0 Validation accuracy: 0.0
(50, 784)
(50,)
4 Train accuracy: 0.0 Validation accuracy: 0.0
(50, 784)
(50,)
5 Train accuracy: 0.0 Validation accuracy: 0.0
(50, 784)
(50,)
6 Train accuracy: 0.0 Validation accuracy: 0.0
(50, 784)
(50,)
7 Train accuracy: 0.0 Validation accuracy: 0.0
(50, 784)
(50,)
8 Train accuracy: 0.0 Validation accuracy: 0.0
(50, 784)
(50,)
9 Train accuracy: 0.0 Validation accuracy: 0.0
(50, 784)
(50,)
10 Train accuracy: 0.0 Validation accuracy: 0.0
(50, 784)
(50,)
11 Train accuracy: 0.0 Validation accuracy: 0.0
(50, 784)
(50,)
12 Train accuracy: 0.0 Validation accuracy: 0.0
(50, 784)
(50,)
13 Train accuracy: 0.0 Validation accuracy: 0.0
(50, 784)
(50,)
14 Train accuracy: 0.0 Validation accuracy: 0.0
(50, 784)
(50,)
15 Train accuracy: 0.0 Validation 

In [27]:
feature_columns = tf.contrib.learn.infer_real_valued_columns_from_input(X_train)
dnn_clf = tf.contrib.learn.DNNClassifier(hidden_units=[300, 100], n_classes=10,
                                         feature_columns=feature_columns)
dnn_clf.fit(x=X_train, y=Y_train, batch_size=50, steps=40000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f03f40dc0b8>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/tmpu1xmlkai'}
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Ex

INFO:tensorflow:global_step/sec: 143.451
INFO:tensorflow:loss = 0.20964505, step = 4001 (0.696 sec)
INFO:tensorflow:global_step/sec: 149.649
INFO:tensorflow:loss = 0.15811272, step = 4101 (0.668 sec)
INFO:tensorflow:global_step/sec: 123.577
INFO:tensorflow:loss = 0.05147886, step = 4201 (0.809 sec)
INFO:tensorflow:global_step/sec: 151.19
INFO:tensorflow:loss = 0.18068768, step = 4301 (0.663 sec)
INFO:tensorflow:global_step/sec: 196.347
INFO:tensorflow:loss = 0.1187226, step = 4401 (0.508 sec)
INFO:tensorflow:global_step/sec: 148.325
INFO:tensorflow:loss = 0.46425, step = 4501 (0.674 sec)
INFO:tensorflow:global_step/sec: 187.73
INFO:tensorflow:loss = 0.16523646, step = 4601 (0.534 sec)
INFO:tensorflow:global_step/sec: 151.928
INFO:tensorflow:loss = 0.074145675, step = 4701 (0.657 sec)
INFO:tensorflow:global_step/sec: 181.677
INFO:tensorflow:loss = 0.19467877, step = 4801 (0.552 sec)
INFO:tensorflow:global_step/sec: 193.741
INFO:tensorflow:loss = 0.14792377, step = 4901 (0.515 sec)
INFO:

INFO:tensorflow:global_step/sec: 190.292
INFO:tensorflow:loss = 0.08862325, step = 12201 (0.526 sec)
INFO:tensorflow:global_step/sec: 207.03
INFO:tensorflow:loss = 0.06449833, step = 12301 (0.483 sec)
INFO:tensorflow:global_step/sec: 190.29
INFO:tensorflow:loss = 0.11932072, step = 12401 (0.530 sec)
INFO:tensorflow:global_step/sec: 189.376
INFO:tensorflow:loss = 0.18756095, step = 12501 (0.524 sec)
INFO:tensorflow:global_step/sec: 200.339
INFO:tensorflow:loss = 0.21754067, step = 12601 (0.499 sec)
INFO:tensorflow:global_step/sec: 198.755
INFO:tensorflow:loss = 0.24985006, step = 12701 (0.503 sec)
INFO:tensorflow:global_step/sec: 195.685
INFO:tensorflow:loss = 0.26731887, step = 12801 (0.513 sec)
INFO:tensorflow:global_step/sec: 184.278
INFO:tensorflow:loss = 0.15033717, step = 12901 (0.541 sec)
INFO:tensorflow:global_step/sec: 198.877
INFO:tensorflow:loss = 0.2419467, step = 13001 (0.503 sec)
INFO:tensorflow:global_step/sec: 172.582
INFO:tensorflow:loss = 0.19910103, step = 13101 (0.58

INFO:tensorflow:global_step/sec: 195.259
INFO:tensorflow:loss = 0.10448585, step = 20301 (0.512 sec)
INFO:tensorflow:global_step/sec: 189.112
INFO:tensorflow:loss = 0.008799132, step = 20401 (0.530 sec)
INFO:tensorflow:global_step/sec: 140.246
INFO:tensorflow:loss = 0.14507665, step = 20501 (0.714 sec)
INFO:tensorflow:global_step/sec: 151.256
INFO:tensorflow:loss = 0.05218003, step = 20601 (0.660 sec)
INFO:tensorflow:global_step/sec: 177.294
INFO:tensorflow:loss = 0.08656195, step = 20701 (0.563 sec)
INFO:tensorflow:global_step/sec: 178.002
INFO:tensorflow:loss = 0.088881195, step = 20801 (0.562 sec)
INFO:tensorflow:global_step/sec: 144.618
INFO:tensorflow:loss = 0.09077127, step = 20901 (0.691 sec)
INFO:tensorflow:global_step/sec: 155.98
INFO:tensorflow:loss = 0.09880128, step = 21001 (0.641 sec)
INFO:tensorflow:global_step/sec: 165.793
INFO:tensorflow:loss = 0.044814423, step = 21101 (0.603 sec)
INFO:tensorflow:global_step/sec: 162.753
INFO:tensorflow:loss = 0.18395133, step = 21201 

INFO:tensorflow:global_step/sec: 178.944
INFO:tensorflow:loss = 0.019988209, step = 28401 (0.558 sec)
INFO:tensorflow:global_step/sec: 150.615
INFO:tensorflow:loss = 0.10325295, step = 28501 (0.676 sec)
INFO:tensorflow:global_step/sec: 117.188
INFO:tensorflow:loss = 0.008281672, step = 28601 (0.842 sec)
INFO:tensorflow:global_step/sec: 142.065
INFO:tensorflow:loss = 0.06467534, step = 28701 (0.704 sec)
INFO:tensorflow:global_step/sec: 189.457
INFO:tensorflow:loss = 0.0070623164, step = 28801 (0.528 sec)
INFO:tensorflow:global_step/sec: 158.033
INFO:tensorflow:loss = 0.055566665, step = 28901 (0.633 sec)
INFO:tensorflow:global_step/sec: 158.048
INFO:tensorflow:loss = 0.20714521, step = 29001 (0.637 sec)
INFO:tensorflow:global_step/sec: 181.686
INFO:tensorflow:loss = 0.020487072, step = 29101 (0.547 sec)
INFO:tensorflow:global_step/sec: 182.608
INFO:tensorflow:loss = 0.12652054, step = 29201 (0.548 sec)
INFO:tensorflow:global_step/sec: 121.257
INFO:tensorflow:loss = 0.010736009, step = 2

INFO:tensorflow:global_step/sec: 110.344
INFO:tensorflow:loss = 0.008549577, step = 36501 (0.911 sec)
INFO:tensorflow:global_step/sec: 119.215
INFO:tensorflow:loss = 0.10798338, step = 36601 (0.835 sec)
INFO:tensorflow:global_step/sec: 146.525
INFO:tensorflow:loss = 0.06055058, step = 36701 (0.681 sec)
INFO:tensorflow:global_step/sec: 121.597
INFO:tensorflow:loss = 0.0421755, step = 36801 (0.824 sec)
INFO:tensorflow:global_step/sec: 148.356
INFO:tensorflow:loss = 0.06841208, step = 36901 (0.673 sec)
INFO:tensorflow:global_step/sec: 178.475
INFO:tensorflow:loss = 0.06943139, step = 37001 (0.562 sec)
INFO:tensorflow:global_step/sec: 175.199
INFO:tensorflow:loss = 0.008046676, step = 37101 (0.571 sec)
INFO:tensorflow:global_step/sec: 169.944
INFO:tensorflow:loss = 0.0030262494, step = 37201 (0.587 sec)
INFO:tensorflow:global_step/sec: 140.457
INFO:tensorflow:loss = 0.07728417, step = 37301 (0.712 sec)
INFO:tensorflow:global_step/sec: 148.563
INFO:tensorflow:loss = 0.031484205, step = 3740

DNNClassifier(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._MultiClassHead object at 0x7f03f43a7358>, 'hidden_units': [300, 100], 'feature_columns': (_RealValuedColumn(column_name='', dimension=784, default_value=None, dtype=tf.float64, normalizer=None),), 'optimizer': None, 'activation_fn': <function relu at 0x7f03ffaec0d0>, 'dropout': None, 'gradient_clip_norm': None, 'embedding_lr_multipliers': None, 'input_layer_min_slice_size': None})

In [28]:
from sklearn.metrics import accuracy_score

y_pred = list(dnn_clf.predict(X_test))
accuracy = accuracy_score(Y_test, y_pred)
accuracy

Instructions for updating:
Please switch to predict_classes, or set `outputs` argument.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpu1xmlkai/model.ckpt-40000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


0.94

In [29]:
submit_test = pd.read_csv("./data/test.csv")

In [30]:
submit_pred = list(dnn_clf.predict(submit_test))

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpu1xmlkai/model.ckpt-40000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [31]:
submit_pred

[2,
 0,
 9,
 9,
 3,
 7,
 0,
 3,
 0,
 3,
 5,
 7,
 4,
 0,
 4,
 3,
 3,
 1,
 9,
 0,
 9,
 1,
 1,
 5,
 7,
 4,
 2,
 7,
 7,
 7,
 7,
 5,
 4,
 2,
 6,
 2,
 5,
 5,
 1,
 6,
 7,
 7,
 4,
 9,
 8,
 7,
 8,
 2,
 6,
 7,
 6,
 8,
 8,
 3,
 8,
 2,
 1,
 2,
 2,
 0,
 7,
 1,
 7,
 0,
 0,
 0,
 1,
 9,
 0,
 1,
 6,
 5,
 8,
 8,
 2,
 8,
 8,
 9,
 2,
 3,
 5,
 9,
 1,
 0,
 9,
 2,
 4,
 3,
 6,
 7,
 2,
 0,
 6,
 6,
 1,
 9,
 3,
 9,
 7,
 4,
 0,
 9,
 2,
 0,
 7,
 3,
 0,
 5,
 0,
 9,
 0,
 0,
 4,
 7,
 1,
 7,
 1,
 1,
 5,
 3,
 3,
 7,
 2,
 8,
 6,
 3,
 8,
 7,
 8,
 4,
 3,
 5,
 6,
 0,
 0,
 0,
 3,
 1,
 5,
 0,
 0,
 3,
 4,
 5,
 5,
 8,
 7,
 7,
 2,
 8,
 4,
 3,
 5,
 6,
 5,
 3,
 7,
 5,
 7,
 8,
 3,
 0,
 4,
 5,
 1,
 2,
 7,
 6,
 3,
 0,
 3,
 7,
 8,
 6,
 1,
 3,
 7,
 7,
 1,
 2,
 4,
 3,
 5,
 2,
 4,
 9,
 2,
 1,
 6,
 0,
 6,
 1,
 4,
 9,
 6,
 0,
 9,
 7,
 6,
 9,
 1,
 9,
 0,
 9,
 9,
 0,
 8,
 4,
 6,
 2,
 0,
 9,
 3,
 6,
 7,
 2,
 1,
 6,
 3,
 4,
 2,
 3,
 1,
 2,
 2,
 8,
 4,
 6,
 1,
 0,
 0,
 4,
 9,
 1,
 7,
 3,
 2,
 3,
 8,
 6,
 8,
 6,
 2,
 8,
 5,
 5,
 4,
 8,
 3,
 8,


In [33]:
n_submit = len(submit_pred)

In [34]:
submit = {'ImageId': [i+1 for i in range(n_submit)], 'Label': submit_pred}

In [35]:
submit.head

AttributeError: 'dict' object has no attribute 'head'

In [36]:
submit_df = pd.DataFrame(data=submit)

In [38]:
submit_df.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


In [39]:
submit_df.to_csv("dnn_predict.csv")